In [1]:
from tensorflow.keras.layers import Dense, BatchNormalization, Conv1DTranspose, Conv1D, Flatten, Add, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras import backend as K
from keras.utils.vis_utils import plot_model

In [2]:
import matplotlib.pyplot as plt
import numpy as np

from commpy.modulation import QAMModem

from optic.dsp import pulseShape, firFilter, decimate, symbolSync, resample
from optic.models import phaseNoise, linFiberCh, KramersKronigRx, photodiode

from optic.tx import simpleWDMTx
from optic.core import parameters
from optic.equalization import edc, mimoAdaptEqualizer
from optic.carrierRecovery import cpr
from optic.metrics import fastBERcalc, monteCarloGMI, monteCarloMI, signal_power
from optic.plot import pconst

import scipy.constants as const
from tqdm.notebook import tqdm
import sys

In [3]:
def loadArray(file_path):
    with open(file_path, 'rb') as stream:
        data = np.load(stream)
    return data

In [53]:
def permutate_input_output(input_matrix, output_matrix):
    
    num_rows, num_cols = input_matrix.shape
    # Cria uma matriz de permutação para trocar as linhas
    permutation = np.random.permutation(num_rows)

    # Aplica a permutação nas matrizes de entrada e saída
    permuted_input_matrix = input_matrix[permutation]
    permuted_output_matrix = output_matrix[permutation]

    return permuted_input_matrix, permuted_output_matrix

In [37]:
def create_convnet(N, n0, img_path='convnet_image.png'):
    
    """ Esta função retorna o modelo da rede neural citada pelo artigo 
    "Deep learning-based Phase Retrieval Scheme for Minimum-phase Signal Recovery".
    Alguns parâmetros podem ser fixos como:
    
    k = 3: tamanho do kernel das camadas convolucionais 1D dentro de um Bloco D/U
    s: passo das camadas convolucionais 1D dentro de um bloco D/U, para este caso foi selecionado s = 2.
    d: profundidade do modelo, ou seja, o número de blocos D e U, para este caso foi selecionado d = 3
        
    Args:
        N (int): Número de amostras de amplitude do sinal.
        n0 (int): Número de núcleos de camadas convolucionais 1D dentro de um bloco D/U
        img_path (str, optional): Nome do arquivo de saída para o plot do modelo em .png. Defaults to 'convnet_image.png'.

    Returns:
        object: modelo da rede neural.
    """
    
    model = Sequential()   
    input_shape = tf.keras.Input(shape=(N, 1))
    
    normalize = BatchNormalization()(input_shape)
    relu_layer = Activation('relu')(normalize)

    # skip connection
    skipConnection0 = Conv1D(filters=n0, kernel_size=1, activation='linear')(relu_layer)
    
    ## D Block 01
    Dtower_7 = Conv1D(n0, kernel_size=3, padding='same', strides=2, activation='relu')(relu_layer)
    # parallel layer
    Dtower_parallel_01 = Conv1D(n0, kernel_size=3, padding='same', strides=2, activation='relu')(relu_layer)
    Dtower_8 = Conv1D(n0, kernel_size=3, padding='same', activation='relu')(Dtower_7)
    D01_added = Add()([Dtower_8, Dtower_parallel_01])
     
    # skip connection
    skipConnection1 = Conv1D(filters=n0, kernel_size=1, activation='linear')(D01_added)
    
    ## D Block 02
    Dtower_5 = Conv1D(n0, kernel_size=3, padding='same', strides=2, activation='relu')(D01_added)
    # parallel layer
    Dtower_parallel_02 = Conv1D(n0, kernel_size=3, padding='same', strides=2, activation='relu')(D01_added)  
    Dtower_6 = Conv1D(n0, kernel_size=3, padding='same', activation='relu')(Dtower_5)
    D02_added = Add()([Dtower_6, Dtower_parallel_02])
    
    # skip connection
    skipConnection2 = Conv1D(filters=n0, kernel_size=1, activation='linear')(D02_added)
    
    ## D Block 03
    Dtower_3 = Conv1D(n0, kernel_size=3, padding='same', strides=2, activation='relu')(D02_added)
    # parallel layer
    Dtower_parallel_03 = Conv1D(n0, kernel_size=3, padding='same', strides=2, activation='relu')(D02_added)  
    Dtower_4 = Conv1D(n0, kernel_size=3, padding='same', activation='relu')(Dtower_3)
    D03_added = Add()([Dtower_4, Dtower_parallel_03])

    ## U Block 03 
    tower_1 = Conv1DTranspose(n0, kernel_size=3, padding='same', strides=2, activation='relu')(D03_added)
    # parallel layer
    tower_parallel_03 = Conv1DTranspose(n0, kernel_size=3, padding='same', strides=2, activation='relu')(D03_added)
    tower_2 = Conv1D(n0, kernel_size=3, padding='same', activation='relu')(tower_1)
    U01_added = Add()([tower_2, tower_parallel_03, skipConnection2])

    ## U Block 02
    tower_3 = Conv1DTranspose(n0, kernel_size=3, padding='same', strides=2, activation='relu')(U01_added)
    # parallel layer
    tower_parallel_02 = Conv1DTranspose(n0, kernel_size=3, padding='same', strides=2, activation='relu')(U01_added) 
    tower_4 = Conv1D(n0, kernel_size=3, padding='same', activation='relu')(tower_3)
    U02_added = Add()([tower_4, tower_parallel_02, skipConnection1])
    
    ## U Block 01
    tower_5 = Conv1DTranspose(n0, kernel_size=3, padding='same', strides=2, activation='relu')(U02_added)
    # parallel layer
    tower_parallel_01 = Conv1DTranspose(n0, kernel_size=3, padding='same', strides=2, activation='relu')(U02_added)  
    tower_6 = Conv1D(n0, kernel_size=3, padding='same', activation='relu')(tower_5)
    U03_added = Add()([tower_6, tower_parallel_01, skipConnection0])
    
    # output layer
    output_layer = Conv1D(2, kernel_size=3, padding='same', activation='linear')(U03_added)
    flat = Flatten()(output_layer)
    output = Dense(2, activation='linear')(flat)
    
    model = tf.keras.Model(input_shape, output)
    plot_model(model, to_file=img_path, show_layer_names=True, show_layer_activations=True)
    
    return model

In [38]:
model = create_convnet(128, 32)
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 128, 1)]     0           []                               
                                                                                                  
 batch_normalization_5 (BatchNo  (None, 128, 1)      4           ['input_6[0][0]']                
 rmalization)                                                                                     
                                                                                                  
 activation_5 (Activation)      (None, 128, 1)       0           ['batch_normalization_5[0][0]']  
                                                                                                  
 conv1d_81 (Conv1D)             (None, 64, 32)       128         ['activation_5[0][0]']     

In [11]:
## Transmitter parameters:
paramTx = parameters()
paramTx.M = 16                 # order of the modulation format
paramTx.Rs = 32e9              # symbol rate [baud]
paramTx.SpS = 4                # samples per symbol
paramTx.Nbits = 3000           # total number of bits per polarization
paramTx.pulse = "rrc"          # pulse shaping filter
paramTx.Ntaps = 1024           # number of pulse shaping filter coefficients
paramTx.alphaRRC = 0.05        # RRC rolloff
paramTx.Pch_dBm = 0            # power of the optical signal [dBm]
paramTx.Nch = 1                # number of WDM channels
paramTx.Fc = 193.1e12          # central frequency of the optical spectrum
paramTx.freqSpac = 37.5e9      # WDM grid spacing

## Optical channel parameters:
Ltotal = 100       # total link distance [km]
alpha = 0        # fiber loss parameter [dB/km]
D = 16           # fiber dispersion parameter [ps/nm/km]
Fc = paramTx.Fc  # central optical frequency of the WDM spectrum [Hz]

## Receiver parameters:

# local oscillator (LO)
FO = paramTx.Rs/2   # frequency offset
lw = 0*200e3        # linewidth
ϕ_lo = 0            # initial phase in rad
# Plo_dBm = 11        # power in dBm outlier

# ADC sampling rate
paramADC = parameters()
paramADC.Rs = paramTx.Rs
paramADC.SpS_in = paramTx.SpS
paramADC.SpS_out = 4

## General simulation parameters:
chIndex = 0  # index of the channel to be demodulated
plotPSD = True
Fs = paramTx.Rs * paramTx.SpS  # simulation sampling rate

# photodiode parameters
paramPD = parameters()
paramPD.B  = 1.1*paramTx.Rs
paramPD.Fs = Fs
Ts = 1 / Fs

In [12]:
path = 'C:/Users/optic/Documents/PIVIC-PIBIC-Comunicacoes-Opticas/models/CONV_models/'
N = 128 # number of input amplitude samples to the NN
n0 = 32

loPower = np.arange(6,16,1)

for indPower, Plo_dBm in enumerate(tqdm(loPower)):

    # generate optical signal signal
    sigTx, symbTx_, paramTx = simpleWDMTx(paramTx)

    # simulate linear signal propagation
    sigCh = linFiberCh(sigTx, Ltotal, alpha, D, Fc, Fs)

    symbTx = symbTx_[:, :, chIndex]
    Plo = 10 ** (Plo_dBm / 10) * 1e-3  # power in W

    # generate LO field
    π = np.pi
    t = np.arange(0, len(sigCh))*Ts
    ϕ_pn_lo = phaseNoise(lw, len(sigCh), Ts)

    sigLO = np.sqrt(Plo) * np.exp(-1j * (2 * π * FO * t + ϕ_lo + ϕ_pn_lo))

    # Add LO to the received signal
    sigRx = np.sqrt(Plo) + sigCh* np.exp(1j * (2 * π * FO * t + ϕ_lo + ϕ_pn_lo))
    sfm   = sigRx.copy()

    print('CSPR = %.2f dB'%(10*np.log10(signal_power(sigLO)/signal_power(sigCh))))

    # simulate ideal direct-detection optical receiver
    Ipd = photodiode(sigRx, paramPD)
    Amp = np.sqrt(Ipd.real)
    Amp = resample(Amp, paramADC).real

    # resampling to ADC sampling rate
    sigCh = resample(sigCh, paramADC)
    sfm = resample(sfm, paramADC)
    newFs = paramADC.SpS_out*paramTx.Rs

    sfm = sfm/np.sqrt(signal_power(sfm))
    
    # Neural network training

    #sigPhase = np.angle(sfm) # get signal phase samples (labels) (L,)
    sigAmp = np.pad(Amp, (int(N/2), int(N/2)), 'constant') # get signal amplitude samples (L,)

    # create set of input features
    X_train = np.zeros((len(sfm), N)) #(L,N)

    for indPhase in range(len(sfm)):
        X_train[indPhase] = sigAmp[indPhase:N+indPhase]

    # create set of phase labels
    y_train = np.array([sfm.real, sfm.imag]).T
    
    np.save(r'C:\Users\Silas\Documents\PIVIC-Comunicacoes-Opticas\resultados\Data\Pibic\training data\X_train_'+str(Plo_dBm)+'dB', X_train)
    np.save(r'C:\Users\Silas\Documents\PIVIC-Comunicacoes-Opticas\resultados\Data\Pibic\training data\y_train_'+str(Plo_dBm)+'dB', y_train)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

CSPR = 6.00 dB


  0%|          | 0/1 [00:00<?, ?it/s]

CSPR = 7.00 dB


...

CSPR = 13.00 dB


  0%|          | 0/1 [00:00<?, ?it/s]

CSPR = 14.00 dB


  0%|          | 0/1 [00:00<?, ?it/s]

CSPR = 15.00 dB


In [13]:
X_train_6dB = loadArray(r'C:\Users\Silas\Documents\PIVIC-Comunicacoes-Opticas\resultados\Data\Pibic\training data\X_train_6dB.npy')
X_train_7dB = loadArray(r'C:\Users\Silas\Documents\PIVIC-Comunicacoes-Opticas\resultados\Data\Pibic\training data\X_train_7dB.npy')
X_train_8dB = loadArray(r'C:\Users\Silas\Documents\PIVIC-Comunicacoes-Opticas\resultados\Data\Pibic\training data\X_train_8dB.npy')
X_train_9dB = loadArray(r'C:\Users\Silas\Documents\PIVIC-Comunicacoes-Opticas\resultados\Data\Pibic\training data\X_train_9dB.npy')
X_train_10dB = loadArray(r'C:\Users\Silas\Documents\PIVIC-Comunicacoes-Opticas\resultados\Data\Pibic\training data\X_train_10dB.npy')
X_train_11dB = loadArray(r'C:\Users\Silas\Documents\PIVIC-Comunicacoes-Opticas\resultados\Data\Pibic\training data\X_train_11dB.npy')
X_train_12dB = loadArray(r'C:\Users\Silas\Documents\PIVIC-Comunicacoes-Opticas\resultados\Data\Pibic\training data\X_train_12dB.npy')
X_train_13dB = loadArray(r'C:\Users\Silas\Documents\PIVIC-Comunicacoes-Opticas\resultados\Data\Pibic\training data\X_train_13dB.npy')
X_train_14dB = loadArray(r'C:\Users\Silas\Documents\PIVIC-Comunicacoes-Opticas\resultados\Data\Pibic\training data\X_train_14dB.npy')
X_train_15dB = loadArray(r'C:\Users\Silas\Documents\PIVIC-Comunicacoes-Opticas\resultados\Data\Pibic\training data\X_train_15dB.npy')

In [14]:
y_train_6dB = loadArray(r'C:\Users\Silas\Documents\PIVIC-Comunicacoes-Opticas\resultados\Data\Pibic\training data\y_train_6dB.npy')
y_train_7dB = loadArray(r'C:\Users\Silas\Documents\PIVIC-Comunicacoes-Opticas\resultados\Data\Pibic\training data\y_train_7dB.npy')
y_train_8dB = loadArray(r'C:\Users\Silas\Documents\PIVIC-Comunicacoes-Opticas\resultados\Data\Pibic\training data\y_train_8dB.npy')
y_train_9dB = loadArray(r'C:\Users\Silas\Documents\PIVIC-Comunicacoes-Opticas\resultados\Data\Pibic\training data\y_train_9dB.npy')
y_train_10dB = loadArray(r'C:\Users\Silas\Documents\PIVIC-Comunicacoes-Opticas\resultados\Data\Pibic\training data\y_train_10dB.npy')
y_train_11dB = loadArray(r'C:\Users\Silas\Documents\PIVIC-Comunicacoes-Opticas\resultados\Data\Pibic\training data\y_train_11dB.npy')
y_train_12dB = loadArray(r'C:\Users\Silas\Documents\PIVIC-Comunicacoes-Opticas\resultados\Data\Pibic\training data\y_train_12dB.npy')
y_train_13dB = loadArray(r'C:\Users\Silas\Documents\PIVIC-Comunicacoes-Opticas\resultados\Data\Pibic\training data\y_train_13dB.npy')
y_train_14dB = loadArray(r'C:\Users\Silas\Documents\PIVIC-Comunicacoes-Opticas\resultados\Data\Pibic\training data\y_train_14dB.npy')
y_train_15dB = loadArray(r'C:\Users\Silas\Documents\PIVIC-Comunicacoes-Opticas\resultados\Data\Pibic\training data\y_train_15dB.npy')

In [51]:
X_train = np.concatenate((X_train_6dB, X_train_7dB, X_train_8dB, X_train_9dB, X_train_10dB, X_train_11dB, X_train_12dB, X_train_13dB, X_train_14dB, X_train_15dB), axis=0)
y_train = np.concatenate((y_train_6dB, y_train_7dB, y_train_8dB, y_train_9dB, y_train_10dB, y_train_11dB, y_train_12dB, y_train_13dB, y_train_14dB, y_train_15dB), axis=0)

In [52]:
print(X_train.shape, y_train.shape)

(30000, 128) (30000, 2)


In [54]:
permuted_X_train, permuted_y_train = permutate_input_output(X_train, y_train)

In [ ]:
# define neural network model
stop = EarlyStopping(monitor='val_loss', patience=5)
model = create_convnet(N, n0)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse')
model.fit(permuted_X_train, permuted_y_train, epochs=200, callbacks=[stop], validation_split=0.3, batch_size=256)
# model.summary()

model.save(path+'testModel_SpS_'+str(paramTx.SpS)+'_CSPR_'+str(Plo_dBm)+'dB_DataTrain', save_format='h5')